In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from numpy.linalg import inv
from numpy.linalg import norm
%matplotlib inline

In [2]:
data = pd.read_csv('Default.csv')
X1 = np.array(data['balance'].values.reshape(-1,1))
X2 = np.array(data['income'].values.reshape(-1,1))
Y = np.array(data['default'].values.reshape(-1,1))
Yp= np.array(data['student'].values.reshape(-1,1))

In [3]:
x = [list(t) for t in zip(X1, X2)]
x = np.array(x)
x=x.reshape(10000,2)
len(x)

10000

In [4]:
x1=[]
x2=[]
for i in range(len(Y)):
    if (Y[i]=='Yes'):
        x1.append(x[i])
    else:
        x2.append(x[i])
x1 = np.array(x1)
x2 = np.array(x2)
print(x1[0])

[ 1486.9981216  17854.39702807]


In [5]:
miu1=np.array([np.sum(x1[:,0])/len(x1),np.sum(x1[:,1])/len(x1)])
miu2=np.array([np.sum(x2[:,0])/len(x2),np.sum(x2[:,1])/len(x2)])
pi1=len(x1)/len(x)
pi2=len(x2)/len(x)
print(miu1)

[ 1747.82168961 32089.14712446]


In [6]:
sigma=(np.sum((x[:]-miu1)**2)+np.sum((x[:]-miu2)**2))/(len(x)-2)
SIGMA = np.cov(x,rowvar=False)
print(SIGMA)

[[ 2.33980187e+05 -9.82142336e+05]
 [-9.82142336e+05  1.77865955e+08]]


In [7]:
def BayessianClass(x,mius,pis,SIGMA,k):
    deltas = []
    pos = 0
    m=0
    for i in range(k):
        miuk = mius[i,:]
        pik = pis[i]
        delta = (x.transpose()/norm(x))@(inv(SIGMA)/norm(inv(SIGMA)))@(miuk/norm(miuk)) - (1/2)*(miuk.transpose()/norm(miuk))@(inv(SIGMA)/norm(inv(SIGMA)))@(miuk/norm(miuk)) + np.log(pik)
        deltas.append(delta)
        if delta > m: 
            m = delta
            pos = i
    return deltas, pos

    

In [8]:
delta, i = BayessianClass(np.array(x[136]),np.array([miu1,miu2]),np.array([pi1,pi2]),SIGMA,2)
print(delta)

[-3.3980527705557724, -0.031055566321074445]


In [9]:
def tresholdear(x,mius,pis,SIGMA,k,Y,tresh):
    Defcorrectos = 0
    Nodefcorrecots = 0
    Defincorrectos = 0
    Nodefincorrecots = 0
    for i in range(len(Y)):
        delta,j = BayessianClass(x[i],mius,pis,SIGMA,k)
        if (delta[0]>np.log(tresh) and Y[i]=='Yes'):
        #if (j==0 and Y[i]=='Yes'):
            Defcorrectos = Defcorrectos +1
        else:
        #elif (j==0 and Y[i]=='No'):
            Nodefincorrecots = Nodefincorrecots +1
        if (delta[1]<(np.log(1-tresh)) and Y[i]=='No'):
        #elif (j==1 and Y[i]=='No'):
            Nodefcorrecots = Nodefcorrecots +1
        else:
        #elif (j==1 and Y[i]=='Yes'):
            Defincorrectos = Defincorrectos +1
    return Defcorrectos,Defincorrectos,Nodefcorrecots,Nodefincorrecots

In [14]:
ErrorTotal = []
DefaultsMal = []
NoDefaultsMal = []
DefaulterBien = []
NoDefaultersBien = []
treshold = np.linspace(0,0.5,50)
for i in range(len(treshold)):
    a,b,c,d = tresholdear(np.array(x),np.array([miu1,miu2]),np.array([pi1,pi2]),SIGMA,2,Y,treshold[i])
    DefaultsMal.append(b/(a+b))
    DefaulterBien.append(a/(a+b))
    NoDefaultsMal.append(d/(c+d))
    NoDefaultersBien.append(c/(c+d))
    ErrorTotal.append((b+d)/(a+b+c+d))


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [10]:
a,b,c,d = tresholdear(np.array(x),np.array([miu1,miu2]),np.array([pi1,pi2]),SIGMA,2,Y,0.2)
print(a+b)
print(a+b+c+d)

10000
20000


In [11]:
plt.plot(treshold,ErrorTotal)
plt.plot(treshold,DefaultsMal)
plt.plot(treshold,NoDefaultsMal)

NameError: name 'treshold' is not defined

In [12]:
TruePositive = DefaulterBien
FalsePositive = NoDefaultsMal
plt.plot(FalsePositive,TruePositive)

NameError: name 'DefaulterBien' is not defined

In [13]:
logreg = LogisticRegression()
def esYes(x):
    if x == 'Yes':
        return 1
    else:
        return 0
Y = np.array((list(map(esYes,Y))))
print(Y)
X_train, X_validation, Y_train, Y_validation = sklearn.model_selection.train_test_split(x, Y, test_size=0.5)
camaleon1 = logreg.fit(X_train, Y_train)
camaleonEntrenado = logreg.predict(X_validation)
camaleonEntrenado1 = logreg.predict(X_train)
ii=(camaleonEntrenado == 1) & (Y_validation ==1)
jj=(camaleonEntrenado == 1) & (Y_validation ==0)
kk=(camaleonEntrenado == 0) & (Y_validation ==1)
ll=(camaleonEntrenado == 0) & (Y_validation ==0)
TruePositive = np.count_nonzero(ii)/(np.count_nonzero(ii)+np.count_nonzero(kk))
FalsePositive = np.count_nonzero(jj)/(np.count_nonzero(jj)+np.count_nonzero(ll))
TrueNegative = np.count_nonzero(kk)/(np.count_nonzero(ii)+np.count_nonzero(kk))
FalseNegative = np.count_nonzero(ll)/(np.count_nonzero(jj)+np.count_nonzero(ll))
print(TruePositive, FalsePositive)
print(TrueNegative,FalseNegative)

[0 0 0 ... 0 0 0]
0.0 0.00020682523267838676
1.0 0.9997931747673217


/srv/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
treshold = np.linspace(0,1,50)
def modelo(x,b0,betas):
    #return ((np.exp(b0+b1*x))/(1+np.exp(b0+b1*x)))
    return ((np.exp(b0+np.dot(x,betas.transpose())))/(1+np.exp(b0 + np.dot(x,betas.transpose()))))
camaleon = logreg.fit(X_train, Y_train)
beta0 = logreg.intercept_
betas = logreg.coef_
y = modelo(X_train,beta0,betas)
tp = []
fp = []
tn = []
fn = []
for i in range(len(treshold)):
    for j in range(len(y)):
        if(y[j]>treshold[i]):
            y[j]=1
        else:
            y[j]=0
    ii=(y == 1) & (Y_train ==1)
    jj=(y == 1) & (Y_train ==0)
    kk=(y == 0) & (Y_train ==1)
    ll=(y == 0) & (Y_train ==0)
    TruePositive = np.count_nonzero(ii)/(np.count_nonzero(ii)+np.count_nonzero(kk))
    FalsePositive = np.count_nonzero(jj)/(np.count_nonzero(jj)+np.count_nonzero(ll))
    TrueNegative = np.count_nonzero(kk)/(np.count_nonzero(ii)+np.count_nonzero(kk))
    FalseNegative = np.count_nonzero(ll)/(np.count_nonzero(jj)+np.count_nonzero(ll))
    tp.append(TruePositive)
    fp.append(FalsePositive)
    tn.append(TrueNegative)
    fn.append(FalseNegative)



recall = TruePositive/(TruePositive+FalseNegative)
precision = TruePositive/(TruePositive+FalsePositive)

/srv/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ZeroDivisionError: float division by zero

In [ ]:
plt.plot(recall,precision)